In [ ]:
pm_OrderDate = ""

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

orderSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("Email", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("Tax", FloatType())
    ])

df = spark.read.load(f'abfss://datalakehouse@synapsedp203dl.dfs.core.windows.net/raw/salesdata_{pm_OrderDate}.csv', format='csv', schema=orderSchema, header = True)
  

In [ ]:
customers = df['CustomerName', 'Email', 'OrderDate']

In [ ]:
from pyspark.sql.functions import split, col

# Create the new FirstName and LastName fields
customer_data_df = customers.withColumn("FirstName", split(col("CustomerName"), " ").getItem(0)).withColumn("LastName", split(col("CustomerName"), " ").getItem(1))

# Remove the CustomerName field
customer_data_df = customer_data_df.drop("CustomerName")

display(customer_data_df)

In [ ]:
customer_data_df = customer_data_df.select('FirstName','LastName','Email', 'OrderDate')
display(customer_data_df)

In [ ]:
customer_data_df.write.mode("overwrite").partitionBy("OrderDate").format("delta").option("path", "abfss://datalakehouse@synapsedp203dl.dfs.core.windows.net/refined/Customers").save()
print ("Transformed data saved!")

##### Transformations for Product data

In [ ]:
df_products = df.select("OrderDate", "Item", "Quantity", "UnitPrice", "Tax")

In [ ]:
productSales = df_products.withColumn("GrossSales", df_products["Quantity"]*df_products["UnitPrice"])\
.withColumn("NetSales",(df_products["Quantity"]*df_products["UnitPrice"]) - df_products["Tax"])\
.groupBy("OrderDate", "Item")\
.agg(sum("Quantity").alias("TotalQuantity"),sum("GrossSales").alias("Gross_Sales"), sum("NetSales").alias("Net_Sales"))
display(productSales)

In [ ]:
productSales = productSales.select("OrderDate", "Item", "TotalQuantity",round("Gross_Sales", 2).alias('Gross_Sales'),round("Net_Sales", 2).alias('Net_Sales'))

In [ ]:
display(productSales)

In [ ]:
productSales.write.mode("overwrite").partitionBy("OrderDate").format("delta").option("path", "abfss://datalakehouse@synapsedp203dl.dfs.core.windows.net/refined/ProductSales").save()
print ("Transformed data saved!")